In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score
import tensorflow as tf 
import os 

In [2]:
data = []  # List to store image data
labels = []  # List to store labels

# Define a function to extract the person label from the file name
def extract_person_label(file_name):
    return int(file_name.split('.')[0].replace('subject', '')) - 1  # Subtract 1 to make labels start from 0

# Load data and labels
# Assuming the dataset files are in the 'dataset' directory
dataset_dir = 'C:/Users/zaema/Downloads/archive/'

for file_name in os.listdir(dataset_dir):
    img = plt.imread(os.path.join(dataset_dir, file_name))
    # print(img.shape)
    data.append(img.flatten())  # Flatten image into a 1D array
    labels.append(extract_person_label(file_name))

data = np.array(data)
labels = np.array(labels)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)


In [ ]:
class MLP:
    def __init__(self, input_size, output_size, hidden_layers=[], lr=0.01):
        self.input_size = input_size
        self.output_size = output_size
        self.hidden_layers = hidden_layers
        self.lr = lr

        # Initialize weights and biases for each layer
        layer_sizes = [input_size] + hidden_layers + [output_size]
        self.weights = [np.random.randn(layer_sizes[i], layer_sizes[i+1]) for i in range(len(layer_sizes)-1)]
        self.biases = [np.zeros(layer_sizes[i+1]) for i in range(len(layer_sizes)-1)]
       

In [ ]:
import numpy as np

class MLP:
    def __init__(self, input_size, output_size, hidden_layers=[], learning_rate=0.01):
        self.input_size = input_size
        self.output_size = output_size
        self.hidden_layers = hidden_layers
        self.learning_rate = learning_rate
        
        # Initialize weights and biases for each layer
        layer_sizes = [input_size] + hidden_layers + [output_size]
        self.weights = [np.random.randn(layer_sizes[i], layer_sizes[i+1]) for i in range(len(layer_sizes)-1)]
        self.biases = [np.zeros(layer_sizes[i+1]) for i in range(len(layer_sizes)-1)]
    
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    
    def sigmoid_derivative(self, x):
        return x * (1 - x)
    
    def forward(self, x):
        activations = [x]
        for i in range(len(self.weights)):
            net_input = np.dot(activations[i], self.weights[i]) + self.biases[i]
            activation = self.sigmoid(net_input)
            activations.append(activation)
        return activations
    
    def fit(self, X, y, epochs=1000):
        for epoch in range(epochs):
            for i in range(len(X)):
                # Forward propagation
                activations = self.forward(X[i])
                outputs = activations[-1]
                
                # Backpropagation
                error = y[i] - outputs
                deltas = [error * self.sigmoid_derivative(outputs)]
                for j in range(len(activations) - 2, 0, -1):
                    delta = np.dot(deltas[-1], self.weights[j].T) * self.sigmoid_derivative(activations[j])
                    deltas.append(delta)
                deltas.reverse()
                
                # Update weights and biases
                for j in range(len(self.weights)):
                    self.weights[j] += np.outer(activations[j], deltas[j]) * self.learning_rate
                    self.biases[j] += deltas[j] * self.learning_rate
    
    def predict(self, X):
        predictions = []
        for x in X:
            activations = self.forward(x)
            predictions.append(activations[-1])
        return np.array(predictions)
    
    def test(self, X, y):
        predictions = self.predict(X)
        accuracy = np.mean(np.argmax(predictions, axis=1) == np.argmax(y, axis=1))
        return accuracy
    
    def tune(self, X_train, y_train, X_val, y_val, hyperparameters):
        best_accuracy = 0
        best_model = None
        
        for params in hyperparameters:
            hidden_layers, learning_rate, epochs = params
            
            # Create a new MLP with the given hyperparameters
            mlp = MLP(self.input_size, self.output_size, hidden_layers, learning_rate)
            
            # Train the model
            mlp.fit(X_train, y_train, epochs=epochs)
            
            # Test the model on the validation data
            accuracy = mlp.test(X_val, y_val)
            
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_model = mlp
        
        return best_model, best_accuracy
